In [2]:
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 
import requests

In [3]:
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()
json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [4]:
elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,0.0,0.0,232223,0,0,-1,1,0,4,0.0,...,538,88,520,51,558,57,263,44,0.00,0.00
1,NaN,NaN,58822,0,0,-1,1,0,2,0.5,...,703,221,361,120,413,140,250,97,0.00,0.00
2,100.0,100.0,153256,0,0,-1,1,0,3,0.5,...,503,273,495,178,538,203,446,145,0.00,0.00
3,NaN,NaN,438098,0,0,-1,1,0,3,3.2,...,134,87,132,56,81,35,361,116,1.05,0.53
4,NaN,NaN,226597,-1,1,-3,3,0,2,3.2,...,333,39,120,41,212,71,32,11,0.92,0.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,NaN,NaN,87835,0,0,-1,1,0,2,-0.7,...,487,108,318,104,403,134,326,123,0.00,0.00
714,NaN,NaN,231480,0,0,0,0,0,2,-1.0,...,394,80,567,218,598,223,644,219,0.00,0.00
715,NaN,NaN,593001,0,0,0,0,0,3,-1.0,...,220,141,680,288,692,295,686,284,0.00,0.00
716,NaN,NaN,531363,0,0,0,0,0,4,-1.0,...,459,81,612,70,638,73,504,73,0.00,0.00
